In [8]:
from time import sleep, strftime
from random import randint
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import smtplib
from email.mime.multipart import MIMEMultipart

In [9]:
driver = webdriver.Chrome()
kayak = 'https://www.kayak.fr/flights/CDG-DXB/2025-03-02/2025-05-04?ucs=1yezklu&sort=bestflight_a'
driver.get(kayak)
sleep(3)

In [ ]:
# Closing the popup

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

try:
    # Cibler le bouton "Tout refuser" et cliquer dessus
    reject_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//button[.//div[contains(text(), "Tout refuser")]]'))
    )
    reject_button.click()
    print("Pop-up fermé avec 'Tout refuser'.")
except Exception as e:
    print(f"Erreur pour cliquer sur 'Tout refuser' : {e}")
finally:
    sleep(3)  
    #driver.quit()


Pop-up fermé avec 'Tout refuser'.


In [11]:
# Load more results to maximize the scraping

def load_more():
    try:
        # Attendre que le bouton "Plus de résultats" soit cliquable
        show_more_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//div[contains(@class, "ULvh-button") and contains(@class, "show-more-button") and text()="Plus de résultats"]'))
        )
        show_more_button.click()
        print("Clic sur 'Plus de résultats' réussi.")
    except:
        pass

#load_more()

In [ ]:
from selenium.webdriver.common.by import By
from time import strftime
import pandas as pd

#Scarpping the page "Le Meilleur Choix"
def page_scrape():
    """Scrape flight information correctly with association between flights, durations, stops, cities, and airlines."""

    # Récupérer tous les conteneurs de vols individuels
    xp_flight_containers = '//div[contains(@class, "nrc6-inner")]'
    flight_containers = driver.find_elements(By.XPATH, xp_flight_containers)

    # Initialiser les listes pour stocker les données
    out_durations, return_durations = [], []
    out_times, return_times = [], []
    out_stops, return_stops = [], []
    out_stop_cities, return_stop_cities = [], []
    out_airlines, return_airlines = [], []
    prices = []

    for container in flight_containers:
        try:
            # Durée des vols aller et retour
            durations = container.find_elements(By.XPATH, './/div[contains(@class, "xdW8-mod-full-airport")]/div[contains(@class, "vmXl-mod-variant-default")]')
            out_durations.append(durations[0].text if len(durations) > 0 else None)
            return_durations.append(durations[1].text if len(durations) > 1 else None)

            # Horaires des vols aller et retour
            times = container.find_elements(By.XPATH, './/div[contains(@class, "vmXl-mod-variant-large")]')
            out_times.append(times[0].text if len(times) > 0 else None)
            return_times.append(times[1].text if len(times) > 1 else None)

            # Escales et villes d'escales pour aller et retour
            stops = container.find_elements(By.XPATH, './/div[contains(@class, "JWEO")]/div[contains(@class, "vmXl-mod-variant-default")]/span[contains(@class, "JWEO-stops-text")]')
            out_stops.append(stops[0].text if len(stops) > 0 else "direct")
            return_stops.append(stops[1].text if len(stops) > 1 else "direct")

            stop_cities = container.find_elements(By.XPATH, './/div[contains(@class, "JWEO")]/div[contains(@class, "c_cgF-mod-variant-full-airport")]/span/span')
            out_stop_cities.append(stop_cities[0].text if len(stop_cities) > 0 else None)
            return_stop_cities.append(stop_cities[1].text if len(stop_cities) > 1 else None)

            # Compagnies aériennes pour aller et retour
            airlines = container.find_elements(By.XPATH, './/div[@class="J0g6-labels-grp"]/div[@class="J0g6-operator-text"]')
            airline_text = airlines[0].text if airlines else None
            if airline_text:
                airline_split = airline_text.split(", ")
                out_airlines.append(airline_split[0] if len(airline_split) > 0 else None)
                return_airlines.append(airline_split[1] if len(airline_split) > 1 else airline_split[0])
            else:
                out_airlines.append(None)
                return_airlines.append(None)

            # Prix
            price_element = container.find_element(By.XPATH, './/div[contains(@class, "f8F1-price-text")]')
            prices.append(price_element.text.replace('€', '').strip() if price_element else None)

        except Exception as e:
            out_durations.append(None)
            return_durations.append(None)
            out_times.append(None)
            return_times.append(None)
            out_stops.append(None)
            return_stops.append(None)
            out_stop_cities.append(None)
            return_stop_cities.append(None)
            out_airlines.append(None)
            return_airlines.append(None)
            prices.append(None)
            print(f"Error processing a container: {e}")

  
    flights_df = pd.DataFrame({
        'Out Duration': out_durations,
        'Return Duration': return_durations,
        'Out Time': out_times,
        'Return Time': return_times,
        'Out Stops': out_stops,
        'Return Stops': return_stops,
        'Out Stop Cities': out_stop_cities,
        'Return Stop Cities': return_stop_cities,
        'Out Airline': out_airlines,
        'Return Airline': return_airlines,
        'Price': prices,
    })

    # Ajouter l'horodatage de scraping
    flights_df['Scraped Timestamp'] = strftime("%Y-%m-%d %H:%M:%S")
    return flights_df


flights_df = page_scrape()
flights_df


,Out Duration,Return Duration,Out Time,Return Time,Out Stops,Return Stops,Out Stop Cities,Return Stop Cities,Out Airline,Return Airline,Price,Scraped Timestamp
0,10h 00min,11h 10min,08:15 – 21:15,11:45 – 20:55,1 escale,1 escale,DOH,DOH,Qatar Airways,Qatar Airways,774,2024-12-08 23:20:45
1,11h 25min,8h 40min,14:10 – 04:35\n+1,06:00 – 12:40,1 escale,1 escale,KWI,KWI,Kuwait Airways,Kuwait Airways,403,2024-12-08 23:20:45
2,6h 40min,7h 20min,23:20 – 09:00\n+1,00:40 – 06:00,direct,direct,None,None,Air France,Air France,853,2024-12-08 23:20:45
3,8h 25min,9h 10min,11:10 – 22:35,23:35 – 06:45\n+1,1 escale,1 escale,BAH,BAH,Gulf Air,Gulf Air,589,2024-12-08 23:20:45
4,9h 50min,8h 40min,21:10 – 10:00\n+1,06:00 – 12:40,1 escale,1 escale,MCT,KWI,Oman Air,Kuwait Airways,569,2024-12-08 23:20:45
5,6h 45min,7h 20min,13:35 – 23:20,00:40 – 06:00,direct,direct,None,None,Air France,Air France,853,2024-12-08 23:20:45
6,9h 55min,10h 30min,15:20 – 04:15\n+1,21:50 – 06:20\n+1,1 escale,1 escale,AMM,AMM,Royal Jordanian,Royal Jordanian,555,2024-12-08 23:20:45
7,6h 40min,7h 10min,10:00 – 19:40,08:20 – 13:30,direct,direct,None,None,Emirates,Emirates,917,2024-12-08 23:20:45
8,8h 25min,11h 25min,11:10 – 22:35,21:20 – 06:45\n+1,1 escale,1 escale,BAH,BAH,Gulf Air,Gulf Air,560,2024-12-08 23:20:45
9,9h 55min,10h 20min,10:50 – 23:45,09:20 – 17:40,1 escale,1 escale,LHR,LHR,British Airways,British Airways,578,2024-12-08 23:20:45


In [21]:
import sqlite3

def save_to_database(df):
    """Sauvegarder les données dans une base SQLite."""
    conn = sqlite3.connect("flights_data.db")
    df.to_sql("flights", conn, if_exists="append", index=False)
    conn.close()


save_to_database(flights_df)
print("Données sauvegardées dans la base SQLite.")


Données sauvegardées dans la base SQLite.


In [2]:
import sqlite3
import csv

database_path = r"C:\Users\jinan\OneDrive\Documents\A5_DIA2\Webscraping\flight-webscrapping\flights-scraper\WebScraping\flights_data.db"

conn = sqlite3.connect(database_path)
cursor = conn.cursor()

cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

# Export en CSV
for table_name in tables:
    table_name = table_name[0]  
    output_file = f"{table_name}.csv"

    cursor.execute(f"SELECT * FROM {table_name}")
    rows = cursor.fetchall()

    columns = [description[0] for description in cursor.description]
    
    with open(output_file, "w", newline="", encoding="utf-8") as csv_file:
        csv_writer = csv.writer(csv_file)
        csv_writer.writerow(columns)  
        csv_writer.writerows(rows)   

    print(f"Table '{table_name}' exportée dans {output_file}")

conn.close()


Table 'flights' exportée dans flights.csv


In [8]:
import pandas as pd
csv_path = r"C:\Users\jinan\OneDrive\Documents\A5_DIA2\Webscraping\flight-webscrapping\flights-scraper\WebScraping\flights.csv"
flights_data = pd.read_csv(csv_path, delimiter=',')
flights_data.head()

,Out Duration,Return Duration,Out Time,Return Time,Out Stops,Return Stops,Out Stop Cities,Return Stop Cities,Out Airline,Return Airline,Price,Scraped Timestamp
0,6h 45min,7h 20min,13:35 – 23:20,00:40 – 06:00,direct,direct,NaN,NaN,Air France,Air France,853.0,2024-12-09 00:42:58
1,9h 55min,10h 20min,10:50 – 23:45,09:20 – 17:40,1 escale,1 escale,LHR,LHR,British Airways,British Airways,578.0,2024-12-09 00:42:58
2,9h 55min,10h 20min,18:45 – 07:40\n+1,09:20 – 17:40,1 escale,1 escale,LHR,LHR,British Airways,British Airways,578.0,2024-12-09 00:42:58
3,10h 10min,10h 20min,18:30 – 07:40\n+1,09:20 – 17:40,1 escale,1 escale,LHR,LHR,British Airways,British Airways,578.0,2024-12-09 00:42:58
4,9h 55min,10h 15min,10:50 – 23:45,12:05 – 20:20,1 escale,1 escale,LHR,LHR,British Airways,British Airways,602.0,2024-12-09 00:42:58
